# DataTables, Indexes, Pandas, and Seaborn

## Some useful (free) resources

Introductory:

* [Getting started with Python for research](https://github.com/TiesdeKok/LearnPythonforResearch), a gentle introduction to Python in data-intensive research.

* [A Whirlwind Tour of Python](https://jakevdp.github.io/WhirlwindTourOfPython/index.html), by Jake VanderPlas, another quick Python intro (with notebooks).

Core Pandas/Data Science books:

* [The Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/), by Jake VanderPlas.

* [Python for Data Analysis, 2nd Edition](http://proquest.safaribooksonline.com/book/programming/python/9781491957653), by  Wes McKinney, creator of Pandas. [Companion Notebooks](https://github.com/wesm/pydata-book)

* [Effective Pandas](https://github.com/TomAugspurger/effective-pandas), a book by Tom Augspurger, core Pandas developer.


Complementary resources:

* [An introduction to "Data Science"](https://github.com/stefanv/ds_intro), a collection of Notebooks by BIDS' [Stéfan Van der Walt](https://bids.berkeley.edu/people/st%C3%A9fan-van-der-walt).

* [Effective Computation in Physics](http://proquest.safaribooksonline.com/book/physics/9781491901564), by Kathryn D. Huff; Anthony Scopatz. [Notebooks to accompany the book](https://github.com/physics-codes/seminar). Don't be fooled by the title, it's a great book on modern computational practices with very little that's physics-specific.


OK, let's load and configure some of our core libraries (as an aside, you can find a nice visual gallery of available matplotlib styles [here](https://tonysyu.github.io/raw_content/matplotlib-style-gallery/gallery.html)).

In [68]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

sns.set()
sns.set_context('talk')
np.set_printoptions(threshold=20, precision=2, suppress=True)
pd.options.display.max_rows = 7
pd.options.display.max_columns = 8
pd.set_option('precision', 2)
# This option stops scientific notation for pandas
# pd.set_option('display.float_format', '{:.2f}'.format)

## Reading in DataFrames from Files

Pandas has a number of very useful file reading tools. You can see them enumerated by typing "pd.re" and pressing tab. We'll be using read_csv today. 

In [69]:
elections = pd.read_csv("elections.csv")
elections # if we end a cell with an expression or variable name, the result will print

,Candidate,Party,%,Year,Result
0,Reagan,Republican,50.7,1980,win
1,Carter,Democratic,41.0,1980,loss
2,Anderson,Independent,6.6,1980,loss
...,...,...,...,...,...
20,Romney,Republican,47.2,2012,loss
21,Clinton,Democratic,48.2,2016,loss
22,Trump,Republican,46.1,2016,win


In [132]:
elections.shape

(23, 5)

In [133]:
elections.size

115

In [134]:
elections.describe()

,%,Year
count,23.00,23.00
mean,42.51,1996.87
std,13.48,11.63
...,...,...
50%,47.20,1996.00
75%,49.95,2006.00
max,58.80,2016.00


We can use the head command to show only a few rows of a dataframe.

In [70]:
elections.head(7)

,Candidate,Party,%,Year,Result
0,Reagan,Republican,50.7,1980,win
1,Carter,Democratic,41.0,1980,loss
2,Anderson,Independent,6.6,1980,loss
3,Reagan,Republican,58.8,1984,win
4,Mondale,Democratic,37.6,1984,loss
5,Bush,Republican,53.4,1988,win
6,Dukakis,Democratic,45.6,1988,loss


There is also a tail command.

In [71]:
elections.tail(7)

,Candidate,Party,%,Year,Result
16,Bush,Republican,50.7,2004,win
17,Obama,Democratic,52.9,2008,win
18,McCain,Republican,45.7,2008,loss
19,Obama,Democratic,51.1,2012,win
20,Romney,Republican,47.2,2012,loss
21,Clinton,Democratic,48.2,2016,loss
22,Trump,Republican,46.1,2016,win


The read_csv command lets us specify a column to use an index. For example, we could have used Year as the index.

In [72]:
elections_year_index = pd.read_csv("elections.csv", index_col = "Year")
elections_year_index.head(5)

,Candidate,Party,%,Result
Year,,,,
1980,Reagan,Republican,50.7,win
1980,Carter,Democratic,41.0,loss
1980,Anderson,Independent,6.6,loss
1984,Reagan,Republican,58.8,win
1984,Mondale,Democratic,37.6,loss


Alternately, we could have used the set_index commmand.

In [73]:
elections_party_index = elections.set_index("Party")
elections_party_index.head(5)

,Candidate,%,Year,Result
Party,,,,
Republican,Reagan,50.7,1980,win
Democratic,Carter,41.0,1980,loss
Independent,Anderson,6.6,1980,loss
Republican,Reagan,58.8,1984,win
Democratic,Mondale,37.6,1984,loss


The set_index command (along with almost all other data frame methods) does not modify the dataframe. That is, the original "elections" is untouched. Note: There is a flag called "inplace" which does modify the calling dataframe.

In [74]:
elections.head() #the index remains unchanged

,Candidate,Party,%,Year,Result
0,Reagan,Republican,50.7,1980,win
1,Carter,Democratic,41.0,1980,loss
2,Anderson,Independent,6.6,1980,loss
3,Reagan,Republican,58.8,1984,win
4,Mondale,Democratic,37.6,1984,loss


In [135]:
original = elections
elections = elections.set_index('Year')

original.head(2)

,Candidate,Party,%,Year,Result
0,Reagan,Republican,50.7,1980,win
1,Carter,Democratic,41.0,1980,loss


In [136]:
elections.head(2)

,Candidate,Party,%,Result
Year,,,,
1980,Reagan,Republican,50.7,win
1980,Carter,Democratic,41.0,loss


In [137]:
elections = original
elections.set_index('Year', inplace=True)
original.head(2)

,Candidate,Party,%,Result
Year,,,,
1980,Reagan,Republican,50.7,win
1980,Carter,Democratic,41.0,loss


In [138]:
elections = pd.read_csv('elections.csv')
elections.head(2)

,Candidate,Party,%,Year,Result
0,Reagan,Republican,50.7,1980,win
1,Carter,Democratic,41.0,1980,loss


By contast, column names MUST be unique. For example, if we try to read in a file for which column names are not unique, Pandas will automatically any duplicates.

## The [] Operator

The DataFrame class has an indexing operator [] that lets you do a variety of different things. If your provide a String to the [] operator, you get back a Series corresponding to the requested label.

In [76]:
elections["Candidate"].head(6)

0      Reagan
1      Carter
2    Anderson
3      Reagan
4     Mondale
5        Bush
Name: Candidate, dtype: object

The [] operator also accepts a list of strings. In this case, you get back a DataFrame corresponding to the requested strings.

In [77]:
elections[["Candidate", "Party"]].head(6)

,Candidate,Party
0,Reagan,Republican
1,Carter,Democratic
2,Anderson,Independent
3,Reagan,Republican
4,Mondale,Democratic
5,Bush,Republican


A list of one label also returns a DataFrame. This can be handy if you want your results as a DataFrame, not a series.

In [78]:
elections[["Candidate"]].head(6)

,Candidate
0,Reagan
1,Carter
2,Anderson
3,Reagan
4,Mondale
5,Bush


Note that we can also use the to_frame method to turn a Series into a DataFrame.

In [79]:
elections["Candidate"].to_frame()

,Candidate
0,Reagan
1,Carter
2,Anderson
...,...
20,Romney
21,Clinton
22,Trump


The [] operator also accepts numerical slices as arguments. In this case, we are indexing by row, not column!

In [80]:
elections[0:3]

,Candidate,Party,%,Year,Result
0,Reagan,Republican,50.7,1980,win
1,Carter,Democratic,41.0,1980,loss
2,Anderson,Independent,6.6,1980,loss


If you provide a single argument to the [] operator, it tries to use it as a name. This is true even if the argument passed to [] is an integer. 

In [81]:
#elections[0] #this does not work, try uncommenting this to see it fail in action, woo

The following cells allow you to test your understanding.

In [82]:
weird = pd.DataFrame({1:["topdog","botdog"], "1":["topcat","botcat"]})
weird

,1,1
0,topdog,topcat
1,botdog,botcat


In [83]:
weird[1] #try to predict the output

0    topdog
1    botdog
Name: 1, dtype: object

In [84]:
weird[["1"]] #try to predict the output

,1
0,topcat
1,botcat


In [85]:
weird[1:] #try to predict the output

,1,1
1,botdog,botcat


## Boolean Array Selection

The `[]` operator also supports array of booleans as an input. In this case, the array must be exactly as long as the number of rows. The result is a filtered version of the data frame, where only rows corresponding to True appear.

In [86]:
elections[[False, False, False, False, False, 
          False, False, True, False, False,
          True, False, False, False, True,
          False, False, False, False, False,
          False, False, True]]

,Candidate,Party,%,Year,Result
7,Clinton,Democratic,43.0,1992,win
10,Clinton,Democratic,49.2,1996,win
14,Bush,Republican,47.9,2000,win
22,Trump,Republican,46.1,2016,win


One very common task in Data Science is filtering. Boolean Array Selection is one way to achieve this in Pandas. We start by observing logical operators like the equality operator can be applied to Pandas Series data to generate a Boolean Array. For example, we can compare the 'Result' column to the String 'win':

In [87]:
elections.head(5)

,Candidate,Party,%,Year,Result
0,Reagan,Republican,50.7,1980,win
1,Carter,Democratic,41.0,1980,loss
2,Anderson,Independent,6.6,1980,loss
3,Reagan,Republican,58.8,1984,win
4,Mondale,Democratic,37.6,1984,loss


In [88]:
iswin = elections['Result'] == 'win'
iswin.head(5)

0     True
1    False
2    False
3     True
4    False
Name: Result, dtype: bool

In [89]:
elections[iswin]

,Candidate,Party,%,Year,Result
0,Reagan,Republican,50.7,1980,win
3,Reagan,Republican,58.8,1984,win
5,Bush,Republican,53.4,1988,win
...,...,...,...,...,...
17,Obama,Democratic,52.9,2008,win
19,Obama,Democratic,51.1,2012,win
22,Trump,Republican,46.1,2016,win


The output of the logical operator applied to the Series is another Series with the same name and index, but of datatype boolean. The entry with index i represents the result of the application of that operator to the entry of the original Series with index i.

In [90]:
elections[elections['Party'] == 'Independent']

,Candidate,Party,%,Year,Result
2,Anderson,Independent,6.6,1980,loss
9,Perot,Independent,18.9,1992,loss
12,Perot,Independent,8.4,1996,loss


In [91]:
elections['Result'].head(5)

0     win
1    loss
2    loss
3     win
4    loss
Name: Result, dtype: object

These boolean Series can be used as an argument to the [] operator. For example, the following code creates a DataFrame of all election winners since 1980.

In [92]:
elections.loc[iswin]

,Candidate,Party,%,Year,Result
0,Reagan,Republican,50.7,1980,win
3,Reagan,Republican,58.8,1984,win
5,Bush,Republican,53.4,1988,win
...,...,...,...,...,...
17,Obama,Democratic,52.9,2008,win
19,Obama,Democratic,51.1,2012,win
22,Trump,Republican,46.1,2016,win


Above, we've assigned the result of the logical operator to a new variable called `iswin`. This is uncommon. Usually, the series is created and used on the same line. Such code is a little tricky to read at first, but you'll get used to it quickly.

In [93]:
elections[elections['Result'] == 'win']

,Candidate,Party,%,Year,Result
0,Reagan,Republican,50.7,1980,win
3,Reagan,Republican,58.8,1984,win
5,Bush,Republican,53.4,1988,win
...,...,...,...,...,...
17,Obama,Democratic,52.9,2008,win
19,Obama,Democratic,51.1,2012,win
22,Trump,Republican,46.1,2016,win


We can select multiple criteria by creating multiple boolean Series and combining them using the `&` operator.

In [94]:
elections[(elections['Result'] == 'win')
          & (elections['%'] < 50)]

,Candidate,Party,%,Year,Result
7,Clinton,Democratic,43.0,1992,win
10,Clinton,Democratic,49.2,1996,win
14,Bush,Republican,47.9,2000,win
22,Trump,Republican,46.1,2016,win


## Loc and ILOC

In [95]:
elections.head(5)

,Candidate,Party,%,Year,Result
0,Reagan,Republican,50.7,1980,win
1,Carter,Democratic,41.0,1980,loss
2,Anderson,Independent,6.6,1980,loss
3,Reagan,Republican,58.8,1984,win
4,Mondale,Democratic,37.6,1984,loss


In [96]:
elections.loc[[0, 1, 2, 3, 4], ['Candidate','Party', 'Year']]

,Candidate,Party,Year
0,Reagan,Republican,1980
1,Carter,Democratic,1980
2,Anderson,Independent,1980
3,Reagan,Republican,1984
4,Mondale,Democratic,1984


Loc also supports slicing (for all types, including numeric and string labels!). Note that the slicing for loc is **inclusive**, even for numeric slices.

In [97]:
elections.loc[0:4, 'Candidate':'Year']

,Candidate,Party,%,Year
0,Reagan,Republican,50.7,1980
1,Carter,Democratic,41.0,1980
2,Anderson,Independent,6.6,1980
3,Reagan,Republican,58.8,1984
4,Mondale,Democratic,37.6,1984


If we provide only a single label for the column argument, we get back a Series.

In [98]:
elections.loc[0:4, 'Candidate']

0      Reagan
1      Carter
2    Anderson
3      Reagan
4     Mondale
Name: Candidate, dtype: object

If we want a data frame instead and don't want to use to_frame, we can provde a list containing the column name.

In [99]:
elections.loc[0:4, ['Candidate']]

,Candidate
0,Reagan
1,Carter
2,Anderson
3,Reagan
4,Mondale


If we give only one row but many column labels, we'll get back a Series corresponding to a row of the table. This new Series has a neat index, where each entry is the name of the column that the data came from.

In [100]:
elections.loc[0, 'Candidate':'Year']

Candidate        Reagan
Party        Republican
%                    51
Year               1980
Name: 0, dtype: object

In [101]:
elections.loc[[0], 'Candidate':'Year']

,Candidate,Party,%,Year
0,Reagan,Republican,50.7,1980


If we omit the column argument altogether, the default behavior is to retrieve all columns. 

In [102]:
elections.loc[[2, 4, 5]]

,Candidate,Party,%,Year,Result
2,Anderson,Independent,6.6,1980,loss
4,Mondale,Democratic,37.6,1984,loss
5,Bush,Republican,53.4,1988,win


Loc also supports boolean array inputs instead of labels. If the arrays are too short, loc assumes the missing values are False.

In [103]:
elections.loc[[True, False, False, True], [True, False, False, True]]

,Candidate,Year
0,Reagan,1980
3,Reagan,1984


In [104]:
elections.loc[[0, 3], ['Candidate', 'Year']]

,Candidate,Year
0,Reagan,1980
3,Reagan,1984


We can use boolean array arguments for one axis of the data, and labels for the other.

In [105]:
elections.loc[[True, False, False, True], 'Candidate':'%']

,Candidate,Party,%
0,Reagan,Republican,50.7
3,Reagan,Republican,58.8


Boolean Series are also boolean arrays, so we can use the Boolean Array Selection from earlier using loc as well.

In [106]:
elections.loc[(elections['Result'] == 'win') & (elections['%'] < 50), 
              'Candidate':'%']

,Candidate,Party,%
7,Clinton,Democratic,43.0
10,Clinton,Democratic,49.2
14,Bush,Republican,47.9
22,Trump,Republican,46.1


Let's do a quick example using data with string-labeled rows instead of integer labeled rows, just to make sure we're really understanding loc.

In [107]:
mottos = pd.read_csv("mottos.csv", index_col = "State")
mottos.head(5)

,Motto,Translation,Language,Date Adopted
State,,,,
Alabama,Audemus jura nostra defendere,We dare defend our rights!,Latin,1923
Alaska,North to the future,—,English,1967
Arizona,Ditat Deus,God enriches,Latin,1863
Arkansas,Regnat populus,The people rule,Latin,1907
California,Eureka (Εὕρηκα),I have found it,Greek,1849


As you'd expect, the rows to extract can be specified using slice notation, even if the rows have string labels instead of integer labels.

In [108]:
mottos.loc['California':'Florida', ['Motto', 'Language']]

,Motto,Language
State,,
California,Eureka (Εὕρηκα),Greek
Colorado,Nil sine numine,Latin
Connecticut,Qui transtulit sustinet,Latin
Delaware,Liberty and Independence,English
Florida,In God We Trust,English


Sometimes students are so used to thinking of rows as numbered that they try the following, which will not work.

In [109]:
mottos_extreme = pd.read_csv("mottos_extreme.csv", index_col='State')
mottos_extreme.loc['California']

,Motto,Translation,Language,Date Adopted
State,,,,
California,Eureka (Εὕρηκα),I have found it,Greek,1849
California,We are the real California,—,English,2006


In [110]:
# mottos_extreme.loc['California':'Delaware']
#did i mess up my experiment or is the answer?

### iloc

loc's cousin iloc is very similar, but is used to access based on numerical position instead of label. For example, to access to the top 3 rows and top 3 columns of a table, we can use [0:3, 0:3]. iloc slicing is **exclusive**, just like standard Python slicing of numerical values.

In [111]:
elections.head(5)

,Candidate,Party,%,Year,Result
0,Reagan,Republican,50.7,1980,win
1,Carter,Democratic,41.0,1980,loss
2,Anderson,Independent,6.6,1980,loss
3,Reagan,Republican,58.8,1984,win
4,Mondale,Democratic,37.6,1984,loss


In [112]:
elections.iloc[0:3, 0:3]

,Candidate,Party,%
0,Reagan,Republican,50.7
1,Carter,Democratic,41.0
2,Anderson,Independent,6.6


In [113]:
mottos.iloc[0:3, 0:3]

,Motto,Translation,Language
State,,,
Alabama,Audemus jura nostra defendere,We dare defend our rights!,Latin
Alaska,North to the future,—,English
Arizona,Ditat Deus,God enriches,Latin


We will use both loc and iloc in the course. Loc is generally preferred for a number of reasons, for example: 

1. It is harder to make mistakes since you have to literally write out what you want to get.
2. Code is easier to read, because the reader doesn't have to know e.g. what column #31 represents.
3. It is robust against permutations of the data, e.g. the social security administration switches the order of two columns.

However, iloc is sometimes more convenient. We'll provide examples of when iloc is the superior choice.

## Handy Properties and Utility Functions for Series and DataFrames

The head, shape, size, and describe methods can be used to quickly get a good sense of the data we're working with. For example:

In [115]:
mottos.head(5)

,Motto,Translation,Language,Date Adopted
State,,,,
Alabama,Audemus jura nostra defendere,We dare defend our rights!,Latin,1923
Alaska,North to the future,—,English,1967
Arizona,Ditat Deus,God enriches,Latin,1863
Arkansas,Regnat populus,The people rule,Latin,1907
California,Eureka (Εὕρηκα),I have found it,Greek,1849


In [116]:
mottos.size

200

The fact that the size is 200 means our data file is relatively small, with only 200 total entries.

In [117]:
mottos.shape

(50, 4)

Since we're looking at data for states, and we see the number 50, it looks like we've mostly likely got a complete dataset that omits Washington D.C. and U.S. territories like Guam and Puerto Rico.

In [118]:
mottos.describe()

,Motto,Translation,Language,Date Adopted
count,50,49,50,50
unique,50,30,8,47
top,Regnat populus,—,Latin,1861
freq,1,20,23,2


Above, we see a quick summary of all the data. For example, the most common language for mottos is Latin, which covers 23 different states. Does anything else seem surprising?

We can get a direct reference to the index using .index.

In [119]:
mottos.index

Index(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado',
       'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
       'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
       'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'],
      dtype='object', name='State')

We can also access individual properties of the index, for example, `mottos.index.name`.

In [120]:
mottos.index.name

'State'

This reflects the fact that in our data frame, the index IS the state!

In [121]:
mottos.head(2)

,Motto,Translation,Language,Date Adopted
State,,,,
Alabama,Audemus jura nostra defendere,We dare defend our rights!,Latin,1923
Alaska,North to the future,—,English,1967


It turns out the columns also have an Index. We can access this index by using `.columns`.

In [122]:
mottos.columns

Index(['Motto', 'Translation', 'Language', 'Date Adopted'], dtype='object')

There are also a ton of useful utility methods we can use with Data Frames and Series. For example, we can create a copy of a data frame sorted by a specific column using `sort_values`.

In [123]:
elections.sort_values('%')

,Candidate,Party,%,Year,Result
2,Anderson,Independent,6.6,1980,loss
12,Perot,Independent,8.4,1996,loss
9,Perot,Independent,18.9,1992,loss
...,...,...,...,...,...
17,Obama,Democratic,52.9,2008,win
5,Bush,Republican,53.4,1988,win
3,Reagan,Republican,58.8,1984,win


As mentioned before, all Data Frame methods return a copy and do **not** modify the original data structure, unless you set inplace to True.

In [124]:
elections.head(5)

,Candidate,Party,%,Year,Result
0,Reagan,Republican,50.7,1980,win
1,Carter,Democratic,41.0,1980,loss
2,Anderson,Independent,6.6,1980,loss
3,Reagan,Republican,58.8,1984,win
4,Mondale,Democratic,37.6,1984,loss


If we want to sort in reverse order, we can set `ascending=False`.

In [125]:
elections.sort_values('%', ascending=False)

,Candidate,Party,%,Year,Result
3,Reagan,Republican,58.8,1984,win
5,Bush,Republican,53.4,1988,win
17,Obama,Democratic,52.9,2008,win
...,...,...,...,...,...
9,Perot,Independent,18.9,1992,loss
12,Perot,Independent,8.4,1996,loss
2,Anderson,Independent,6.6,1980,loss


We can also use `sort_values` on Series objects.

In [126]:
mottos['Language'].sort_values().head(10)

State
Washington      Chinook Jargon
Wyoming                English
New Jersey             English
                     ...      
Pennsylvania           English
Rhode Island           English
South Dakota           English
Name: Language, Length: 10, dtype: object

For Series, the `value_counts` method is often quite handy.

In [127]:
elections['Party'].value_counts()

Republican     10
Democratic     10
Independent     3
Name: Party, dtype: int64

In [128]:
mottos['Language'].value_counts()

Latin       23
English     21
Hawaiian     1
            ..
French       1
Italian      1
Greek        1
Name: Language, Length: 8, dtype: int64

Also commonly used is the `unique` method, which returns all unique values as a numpy array.

In [129]:
mottos['Language'].unique()

array(['Latin', 'English', 'Greek', 'Hawaiian', 'Italian', 'French',
       'Spanish', 'Chinook Jargon'], dtype=object)

## Case Study: What was the most popular name in California last year (2019)?

https://www.ssa.gov/OACT/babynames/index.html

https://www.ssa.gov/data

We can download data from the internet with Python, and do so only if needed:

In [25]:
import requests
from pathlib import Path

namesbystate_path = Path('namesbystate.zip')
data_url = 'https://www.ssa.gov/oact/babynames/state/namesbystate.zip'

if not namesbystate_path.exists():
    print('Downloading...', end=' ')
    resp = requests.get(data_url)
    with namesbystate_path.open('wb') as f:
        f.write(resp.content)
    print('Done!')

Let's use Python to understand how this data is laid out:

In [26]:
import zipfile
zf = zipfile.ZipFile(namesbystate_path, 'r')
print([f.filename for f in zf.filelist])

['AK.TXT', 'AL.TXT', 'AR.TXT', 'AZ.TXT', 'CA.TXT', 'CO.TXT', 'CT.TXT', 'DC.TXT', 'DE.TXT', 'FL.TXT', 'GA.TXT', 'HI.TXT', 'IA.TXT', 'ID.TXT', 'IL.TXT', 'IN.TXT', 'KS.TXT', 'KY.TXT', 'LA.TXT', 'MA.TXT', 'MD.TXT', 'ME.TXT', 'MI.TXT', 'MN.TXT', 'MO.TXT', 'MS.TXT', 'MT.TXT', 'NC.TXT', 'ND.TXT', 'NE.TXT', 'NH.TXT', 'NJ.TXT', 'NM.TXT', 'NV.TXT', 'NY.TXT', 'OH.TXT', 'OK.TXT', 'OR.TXT', 'PA.TXT', 'RI.TXT', 'SC.TXT', 'SD.TXT', 'StateReadMe.pdf', 'TN.TXT', 'TX.TXT', 'UT.TXT', 'VA.TXT', 'VT.TXT', 'WA.TXT', 'WI.TXT', 'WV.TXT', 'WY.TXT']


We can pull the PDF readme to view it, but let's operate with the rest of the data in its compressed state:

In [27]:
zf.extract('StateReadMe.pdf')

'/Users/sam/repos/ds100/su19-public/lec/lec02/StateReadMe.pdf'

Let's have a look at the California data, it should give us an idea about the structure of the whole thing:

In [28]:
ca_name = 'CA.TXT'
with zf.open(ca_name) as f:
    for i in range(10):
        print(f.readline().rstrip().decode())

CA,F,1910,Mary,295
CA,F,1910,Helen,239
CA,F,1910,Dorothy,220
CA,F,1910,Margaret,163
CA,F,1910,Frances,134
CA,F,1910,Ruth,128
CA,F,1910,Evelyn,126
CA,F,1910,Alice,118
CA,F,1910,Virginia,101
CA,F,1910,Elizabeth,93


This is equivalent (on macOS or Linux) to extracting the full `CA.TXT` file to disk and then using the `head` command (if you're on Windows, don't try to run the cell below):

In [29]:
zf.extract(ca_name)
!head {ca_name}

In [30]:
!echo {ca_name}

CA.TXT


A couple of practical comments:

* The above is using special tricks in IPython that let you call operating system commands via `!cmd`, and that expand Python variables in such commands with the `{var}` syntax. You can find more about IPython's special tricks [in this tutorial](https://github.com/ipython/ipython-in-depth/blob/master/examples/IPython%20Kernel/Beyond%20Plain%20Python.ipynb).

* `head` doesn't work on Windows, though there are equivalent Windows commands. But by using Python code, even if it's a little bit more verbose, we have a 100% portable solution.

* If the `CA.TXT` file was huge, it would be wasteful to write it all to disk only to look at the start of the file.

The last point is an important, and general theme of this course: we need to learn how to operate with data only on an as-needed basis, because there are many situations in the real world where we can't afford to brute-force 'download all the things'.

Let's remove the `CA.TXT` file to make sure we keep working with our compressed data, as if we couldn't extract it:

In [31]:
import os; os.unlink(ca_name)

## Question 1: What was the most popular name in CA last year?

In [32]:
import pandas as pd

field_names = ['State', 'Sex', 'Year', 'Name', 'Count']
with zf.open(ca_name) as fh:
    ca = pd.read_csv(fh, header=None, names=field_names)
ca.head()

,State,Sex,Year,Name,Count
0,CA,F,1910,Mary,295
1,CA,F,1910,Helen,239
2,CA,F,1910,Dorothy,220
3,CA,F,1910,Margaret,163
4,CA,F,1910,Frances,134


### Indexing Review

Let's play around a bit with our indexing techniques from earlier today.

In [33]:
ca['Count'].head()

0    295
1    239
2    220
3    163
4    134
Name: Count, dtype: int64

In [34]:
ca[0:3]

,State,Sex,Year,Name,Count
0,CA,F,1910,Mary,295
1,CA,F,1910,Helen,239
2,CA,F,1910,Dorothy,220


In [35]:
#ca[0]

In [36]:
ca.iloc[:3, -2:]

,Name,Count
0,Mary,295
1,Helen,239
2,Dorothy,220


In [37]:
ca.loc[0:3, 'State']

0    CA
1    CA
2    CA
3    CA
Name: State, dtype: object

In [38]:
ca['Name'].head()

0        Mary
1       Helen
2     Dorothy
3    Margaret
4     Frances
Name: Name, dtype: object

In [39]:
ca[['Name']].head()

,Name
0,Mary
1,Helen
2,Dorothy
3,Margaret
4,Frances


In [40]:
ca[ca['Year'] == 2017].tail()

,State,Sex,Year,Name,Count
374629,CA,M,2017,Zeth,5
374630,CA,M,2017,Zeyad,5
374631,CA,M,2017,Zia,5
374632,CA,M,2017,Ziad,5
374633,CA,M,2017,Ziv,5


## Understanding the Data

In [41]:
ca.head()

,State,Sex,Year,Name,Count
0,CA,F,1910,Mary,295
1,CA,F,1910,Helen,239
2,CA,F,1910,Dorothy,220
3,CA,F,1910,Margaret,163
4,CA,F,1910,Frances,134


We can get a sense for the shape of our data:

In [42]:
ca.shape

(374634, 5)

In [43]:
ca.size  # rows x columns

1873170

Pandas will give us a summary overview of the *numerical* data in the DataFrame:

In [44]:
ca.describe()

,Year,Count
count,374634.00,374634.00
mean,1982.74,81.49
std,26.11,302.15
...,...,...
50%,1989.00,13.00
75%,2004.00,39.00
max,2017.00,8263.00


And let's look at the *structure* of the DataFrame:

In [45]:
ca.index

RangeIndex(start=0, stop=374634, step=1)

### Sorting

What we've done so far is NOT exploratory data analysis. We were just playing around a bit with the capabilities of the pandas library. Now that we're done, let's turn to the problem at hand: Identifying the most common name in California last year.

In [46]:
ca2017 = ca[ca['Year'] == 2017]
ca_sorted = ca2017.sort_values('Count', ascending=False).head(10)
ca_sorted

,State,Sex,Year,Name,Count
217344,CA,F,2017,Emma,2726
217345,CA,F,2017,Mia,2588
371716,CA,M,2017,Noah,2511
...,...,...,...,...,...
371718,CA,M,2017,Liam,2180
371719,CA,M,2017,Ethan,2141
371720,CA,M,2017,Matthew,2120
